# Extract text and images (Classical Chinese)

This file identifies the text lines manually labelled in Transkribus. It saves individual lines as text and images to different files, which can then be used to create a HF dataset.

In [2]:
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import pandas as pd
import glob
import itertools
import unicodedata
import os 

In [13]:
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

Current working directory: c:\Users\alexm\NUS Dropbox\Alexander Mozdzen\ocr\gongguan-ocr-1\src


In [14]:
output_folder = "../data"
input_folders = [
    f"{output_folder}/1_11",
    f"{output_folder}/12_21", 
    f"{output_folder}/22_31",
    f"{output_folder}/32_41"
]
text_output_folder = f"{output_folder}/texts"
image_output_folder = f"{output_folder}/images"

In [15]:
def normalize_chinese_text(text):
    """
    Normalize text to standard Chinese Unicode form.
    Converts variant Unicode characters (e.g., Kangxi Radicals) into normal forms.
    """
    return unicodedata.normalize("NFKC", text)

New version that uses the included index in the XML file for the reading order of the text lines

In [7]:
#parse XML with index ordering using list comprehension
def parse_xml_index_ordering(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    namespace = {'ns': 'http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15'}

    regions = []

    # Iterate over TextRegion elements
    for text_region in root.findall(".//ns:TextRegion", namespace):
        region_id = text_region.get("id")
        coords_elem = text_region.find("ns:Coords", namespace)

        if coords_elem is None:
            continue

        # Extract TextLines with their indices using list comprehension
        text_lines = [
            (normalize_chinese_text(line.find(".//ns:Unicode", namespace).text),  # Normalized text content
             int(line.get("custom").split("{index:")[1].split(";}")[0]))  # The index number
            for line in text_region.findall(".//ns:TextLine", namespace)
            if line.find(".//ns:Unicode", namespace) is not None
        ]
        
        if not text_lines:
            continue  # Skip empty regions
        
        # Sort text lines by index
        text_lines.sort(key=lambda x: x[1])
        
        # Extract just the text in order
        ordered_text = " ".join(text for text, _ in text_lines)
        
        # Extract coordinates
        coords_str = coords_elem.get("points")
        regions.append((region_id, coords_str, ordered_text))

    return regions

In [10]:
# Draw each text line with buffer and save as image
def create_images_from_regions(page_name, image_path, regions, buffer_above=10, buffer_below=10, buffer_left=10, buffer_right=10):
    data = []
    image = Image.open(image_path)

    for idx, (region_id, coords_str, region_text) in enumerate(regions):
        # Parse the coordinates and find the bounding box
        points = [tuple(map(int, point.split(','))) for point in coords_str.split()]
        
        x_coords = [p[0] for p in points]
        y_coords = [p[1] for p in points]

        min_x, max_x = min(x_coords) - buffer_left, max(x_coords) + buffer_right
        min_y, max_y = min(y_coords) - buffer_above, max(y_coords) + buffer_below

        # Crop the image to the bounding box
        cropped_image = image.crop((min_x, min_y, max_x, max_y))

        # Append the identifier and text to the data list
        data.append([region_text, f'{page_name}_{region_id}'])

        # Save text data to a CSV file
        df = pd.DataFrame(data, columns=['text', 'identifier'])
        df.to_csv(f'{text_output_folder}/{page_name}.csv', index=False)

        # Save the cropped image
        cropped_image.save(f'{image_output_folder}/{page_name}_{region_id}.png')

The following code extracts the coordinates of the text regions from the xml files and crops them into individual images.

In [12]:
for xml_file_path in glob.glob(f"{input_folder}/page/*.xml"):
    # Normalize path and convert backslashes to forward slashes
    xml_file_path = os.path.normpath(xml_file_path).replace("\\", "/")
    page_name = xml_file_path.split("/")[-1].replace(".xml", "")
    image_file_path = f"{input_folder}/{page_name}.jpg" 

    # Parse XML and extract text regions
    regions = parse_xml_index_ordering(xml_file_path)

    # Create images from the text regions
    create_images_from_regions(page_name, image_file_path, regions, buffer_above=0, buffer_below=0, buffer_left=0, buffer_right=0)
